In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib import rcParams
import scipy.constants as sc
import pandas as pd
from numba import njit
from astropy.io import fits
import os
rcParams["figure.figsize"] = (5,2)

In [2]:
@njit
def vel_gen(St0, u0, v0, dz, z_list, a):
    f_loc = v0**2/2
    f_list = [f_loc]
    u = u0 *  np.exp( 1/a * (1 - ( 1 + a*z_list**2 )**(-0.5) ) )
    
    for i in range(1, len(z_list)):
        v_loc = (2*f_loc)**0.5
        f_loc += dz * ( u0/St0 - v_loc/St0 * np.exp( -1/a * (1 - ( 1 + a*z_list[i-1]**2 )**(-0.5) ) ) - z_list[i-1] / (1 + a*z_list[i-1]**2)**(1.5))
        f_list.append(f_loc)
    
    f_list = np.array(f_list)
    v_list = ( 2*f_list )**0.5
    
    return z_list[:len(v_list)], v_list, u[:len(v_list)]

@njit
def vel_loop(St0, u0, v0, a, order, max_list_size, z_start, z_end):
    dz = 10**( np.log10(St0 * u0) - order )
    list_length = (z_end - z_start) / dz
    iter_total = np.ceil(list_length / max_list_size)
    print('Step size:', dz)
    print('Number of iterations:', iter_total)
        
    z_end_loc = z_start + dz * max_list_size
    v_loc = v0
    z_list = []
    v_list = []
    u_list = []

    for i in range(iter_total):
        z_list_loc = np.arange(z_start, z_end_loc, dz)
        z_list_loc, v_list_loc, u_list_loc = vel_gen(St0, u0, v_loc, dz, z_list_loc, a)
        
        z_start = z_end_loc + dz
        z_end_loc += dz * max_list_size + dz
        v_loc = v_list_loc[-1]
        
        z_list.extend(z_list_loc[::int(10*iter_total)])
        v_list.extend(v_list_loc[::int(10*iter_total)])
        u_list.extend(u_list_loc[::int(10*iter_total)])

        if (np.isnan(v_loc)):
            print('Number of actual iterations:', i)
            break
        
    return z_list, v_list, u_list

In [4]:
R = 1 # in au
a = 3e-5 # cm
v0 = 0.
order = 2
max_array_size = 5e8

Omega = (np.sqrt(sc.G * 2e30) * 1.5e11**(-1.5)) * R**(-1.5)
c_s = 1.5 * R**(-0.25) # in km/s
sdot = 2e-12 * (R/10)**(-1.5) # in g/cm2/s
sd = 30 * (1/R) # in g/cm2
rhos = 3.5 # in g/cm3

H_R = c_s / Omega / (1.5e8) / R
eta = H_R**2
St0 = np.sqrt(2*np.pi)* a * rhos / sd
u0 = np.sqrt(2*np.pi) / Omega * sdot / sd

Stb = St0 / u0
H_c_s = 5e6 / (3600*24*365) * R**1.5 # in yr

print('eta:', eta)
print('St_0:', St0)
print('St_b:', Stb)
print('u0:', u0)
print('max_z:', 1/Stb)
print('T:', H_c_s)

print('Array size:', f'{10**order / (St0**2):.2e}')
print('No iterations:', np.ceil(10**order / (St0**2) / max_array_size))

eta: 0.002528355033486658
St_0: 8.7731989612085e-06
St_b: 0.33017202586934385
u0: 2.657159987466728e-05
max_z: 3.02872418511834
T: 0.15854895991882292
Array size: 1.30e+12
No iterations: 2599.0


In [5]:
zl, vl, ul = vel_loop(St0, u0, v0, eta, order, max_array_size, 0, 10)

Step size: 2.3311793241807872e-12
Number of iterations: 8580.0
Number of actual iterations: 2696


In [6]:
data_fits = fits.PrimaryHDU(data=np.array([zl[::10], vl[::10], ul[::10]]))
data_fits.header['a'] = a
data_fits.header['a_units'] = 'cm'
data_fits.header['R'] = R
data_fits.header['R_units'] = 'au'
data_fits.header['ST0'] = St0
data_fits.header['STB'] = Stb
hdul = fits.HDUList([data_fits])
hdul.writeto('/data/jhyl3/vel_fits/3-5cm_radial/r'+str(R)+'a3-5cm.fits', overwrite=True)

In [ ]:
zl, vl, ul = vel_loop(St0, u0, v0, eta, order, 1e5, 0, 4e-6)

In [ ]:
plt.plot(zl[:1000], vl[:1000])
plt.plot(zl[:1000], ul[:1000], c='gray', ls='--', lw=0.8)
plt.xlim(0, 1e-6)

In [ ]:
data_fits = fits.PrimaryHDU(data=np.array([zl[:1000], vl[:1000], ul[:1000]]))
data_fits.header['a'] = a
data_fits.header['a_units'] = 'cm'
data_fits.header['R'] = R
data_fits.header['R_units'] = 'au'
data_fits.header['ST0'] = St0
data_fits.header['STB'] = Stb
hdul = fits.HDUList([data_fits])
hdul.writeto('/data/jhyl3/vel_fits/r'+str(R)+'a1-4cm_initial.fits', overwrite=True)

In [ ]:
plt.plot(zl[::1000], vl[::1000])
plt.plot(zl[::10000], ul[::10000], c='gray', ls='--', lw=0.8)
plt.ylim(0,np.nanmax(vl[::1000])*1.1)

In [ ]:
plt.plot(zl[::1000], vl[::1000])
plt.plot(zl[::10000], ul[::10000], c='gray', ls='--', lw=0.8)
plt.plot(zl2[::10000], ul2[::10000], c='red', ls='--', lw=0.8)
plt.plot(zl2[::1000], vl2[::1000], c='C0')
plt.axvline(1/Stb, c='C3',ls=':', lw=0.8)
plt.ylim(0, 20)
plt.ylabel('$u$ / $c_s$')
plt.xlim(0,20)
plt.xlabel('$z$ / $H$')

### Cell graveyard

In [ ]:
zl2, vl2, ul2 = vel_loop(St0, u0, vl[-1], eta, order, max_array_size, zl[-1])

In [ ]:
zlc = zl + zl2
vlc = vl + vl2
ulc = ul + ul2

In [ ]:
np.array([zlc[::10], vlc[::10], ulc[::10]]).nbytes / 1e6

In [ ]:
data_fits = fits.PrimaryHDU(data=np.array([zlc[::10], vlc[::10], ulc[::10]]))
data_fits.header['a'] = a
data_fits.header['a_units'] = 'cm'
data_fits.header['R'] = R
data_fits.header['R_units'] = 'au'
data_fits.header['ST0'] = St0
data_fits.header['STB'] = Stb
hdul = fits.HDUList([data_fits])
hdul.writeto('/data/jhyl3/vel_fits/r'+str(R)+'a-5cm.fits', overwrite=True)